In [94]:
import tensorflow as tf
import os

In [22]:
daily_sales_numbers = [21,22, -108, 31, -1, 32, 34, 31,90]

In [23]:
tf_dataset=tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [26]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31
90


In [28]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31
90


In [29]:
for sales in tf_dataset.take(4):
    print(sales.numpy())

21
22
-108
31


In [53]:
tf_dataset = tf_dataset.filter(lambda x:x>0)
print([sales for sales in tf_dataset.as_numpy_iterator()])

[21, 22, 31, 32, 34, 31, 90]


In [51]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31
90


In [54]:
tf_dataset = tf_dataset.map(lambda x:x*72)        # to convert to our currency
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232
6480


In [55]:
tf_dataset = tf_dataset.shuffle(3)   # buffer is 3  [x,y,z] choosing a random value from list
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
2304
2232
2448
6480
2232
1512


In [62]:
for sales in tf_dataset.batch(3):
    print(sales.numpy())

[1584 2304 2448]
[2232 6480 2232]
[1512]


In [65]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset.filter(lambda x:x>0). map(lambda y:y*72).shuffle(3).batch(3)

<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [68]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset = tf_dataset.filter(lambda x:x>0). map(lambda y:y*72).shuffle(3).batch(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[2232 2304 1584]
[2448 1512 2232]
[6480]


In [73]:
images_ds = tf.data.Dataset.list_files('images/*/*',shuffle=True)

for file in images_ds.take(5):
    print(file.numpy())
    

b'images\\cats\\download.jpg'
b'images\\cats\\images (2).jpg'
b'images\\dogs\\download (1).jpg'
b'images\\dogs\\download (2).jpg'
b'images\\cats\\images.jpg'


In [74]:
class_names = ["cat","dog"]

In [75]:
image_count = len(images_ds)
image_count

12

In [79]:
train_size = int(image_count*0.8)

train_ds = images_ds.take(train_size)        # take 80% of data for training
test_ds  = images_ds.skip(train_size)        # tak2 20% of data for testing

In [82]:
len(test_ds)

3

In [81]:
len(train_ds)

9

In [85]:
#  retreive the labels

s = "images\\cats\\images.jpg"
s.split("\\")[-2]



'cats'

In [98]:
def get_label(file_path):
    return tf.strings.split(file_path, os.path.sep)[-2]

In [103]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    
    return img, label

In [104]:
for t in train_ds.take(4):
    print(t.numpy())

b'images\\dogs\\download (2).jpg'
b'images\\dogs\\download.jpg'
b'images\\cats\\download (3).jpg'
b'images\\dogs\\download (1).jpg'


In [105]:
for i in train_ds.map(get_label):
    print(i)

tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'dogs', shape=(), dtype=string)
tf.Tensor(b'dogs', shape=(), dtype=string)
tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'cats', shape=(), dtype=string)
tf.Tensor(b'dogs', shape=(), dtype=string)


In [110]:
train_ds = train_ds.map(process_image)
for image, label in train_ds.take(3):
    print("image: ", image)
    print("label: ",label)

image:  tf.Tensor(
[[[ 49.366333 117.47757   31.72763 ]
  [ 67.194244 121.131744  57.40909 ]
  [ 73.60867  113.69461   70.60867 ]
  ...
  [ 30.699219 109.69922   26.699219]
  [ 29.699219 108.69922   25.699219]
  [ 29.021515 108.021515  25.021515]]

 [[ 52.558014 122.193054  39.134277]
  [ 54.856506 107.89166   47.575256]
  [117.59912  157.68506  116.40381 ]
  ...
  [ 27.292969 106.29297   23.292969]
  [ 26.638397 105.6384    22.638397]
  [ 26.195312 105.19531   22.195312]]

 [[ 58.575073 127.76648   50.31906 ]
  [ 69.60727  123.46332   65.375885]
  [103.19797  141.31635  101.34607 ]
  ...
  [ 23.450409 100.67969   19.7984  ]
  [ 22.950714  99.984375  18.878906]
  [ 22.146332  98.984375  17.957031]]

 ...

 [[242.27164  242.27164  242.27164 ]
  [241.1211   241.1211   241.1211  ]
  [239.35037  239.35037  239.35037 ]
  ...
  [186.       219.       148.      ]
  [185.38281  218.38281  147.38281 ]
  [184.46094  217.46094  146.46094 ]]

 [[241.90234  241.90234  241.90234 ]
  [240.55692  240.

In [111]:
def scale(image, label):
    return image/255, label

In [113]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(5):
    print("****Image: ", image.numpy()[0][0])
    print("****Label: ", label.numpy())

****Image:  [0.00051008 0.00052184 0.0004558 ]
****Label:  b'cats'
****Image:  [0.00198703 0.00171021 0.00100279]
****Label:  b'dogs'
****Image:  [0.00312892 0.00240612 0.00157567]
****Label:  b'cats'
****Image:  [0.00390619 0.00390619 0.00390619]
****Label:  b'cats'
****Image:  [0.00112168 0.00185758 0.00061743]
****Label:  b'dogs'
